In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#| default_exp langchain.openassistant
from nbdev.showdoc import show_doc

In [ ]:
from ml.langchain.huggingface import HfInference

In [ ]:
from langchain import PromptTemplate, LLMChain

model_name = "OpenAssistant/oasst-sft-1-pythia-12b"
llm = HfInference(model_name=model_name)

In [ ]:
from getpass import getpass

serper_api_key = getpass('Input password: ')

Input password: ········


In [ ]:
# !pip install gradio

In [ ]:
from langchain import HuggingFacePipeline, LLMMathChain
from langchain.agents import  Tool, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.utilities import GoogleSerperAPIWrapper, PythonREPL
import gradio as gr
import datetime
import os


llm_math_chain = LLMMathChain(llm=llm, verbose=True)
search = GoogleSerperAPIWrapper(serper_api_key=serper_api_key)
python_repl = PythonREPL()

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world\nDO NOT USE THIS FOR SIMPLE QUESTIONS!"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about mathematically related topics\nUSE ONLY FOR MATH RELATED QUESTIONS!"
    ),
    Tool(
        name="Python REPL",
        func=python_repl.run,
        description = "A Python shell. Use this to execute python commands. "
        "Input should be a valid python command. "
        "If you want to see the result, you should print it out "
        "with `print(...)`."
    )
]
tool_names = [tool.name for tool in tools]

C:\local\Anaconda3\envs\py310\lib\site-packages\langchain\chains\llm_math\base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [ ]:
PREFIX = """Consider that you are AI Assistant named AI, AI is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
You love to answer questions and you are very good at it.
Assistant has access to the following tools:"""

In [ ]:
INSTRUCTIONS = """
To use a tool, please use the following format:
``
Thought: Should I use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
``
When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
``
Thought: Should I use a tool? No
AI: [your response here]
``
"""

In [ ]:
SUFFIX = """
CHAT HISTORY:
{chat_history}

Current time: {current_time}
Knowledge date cutoff: 2021-09-01

When answering a question, you MUST use the following language: {language}
Begin!
Question: {input}
Thought: Should I use a tool?{agent_scratchpad}"""

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history",# return_messages=True,
                                  input_key="input", 
                                  output_key='output', 
                                  ai_prefix='AI', 
                                  human_prefix='User')

agent = initialize_agent(tools,
                         llm,
                         agent="conversational-react-description",
                         verbose=True,
                         memory=memory,
                         return_intermediate_steps=False,
                         agent_kwargs={'input_variables': ['input', 'agent_scratchpad', 'chat_history', 'current_time', 'language'],
                                       'prefix': PREFIX,
                                       'format_instructions': INSTRUCTIONS,
                                       'suffix': SUFFIX})
agent.agent.llm_chain.verbose=True

In [ ]:
def main():
    depth = 0
    with gr.Blocks() as app:
        chatbot = gr.Chatbot(elem_id="chatbot")
        state = gr.State([])

        with gr.Row():
            msg = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)

        def user(user_message, history):
            if history is None:
                history = ""
            return "", history + [[user_message, None]]

        def bot(history):
            nonlocal depth
            depth += 1
            prompt = history[-1][0]
            res = agent({"input":prompt,
                         "current_time":datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                         "language": "English"})

            response = res['output']
            history[-1][1] = response

            # free up some memory if we have too many messages
            if depth > 8 or len(memory.buffer) > 512:
                memory.chat_memory.messages.pop(0)
            return history

        msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
                bot, chatbot, chatbot)

    app.queue()
    app.launch(share=False, server_name="0.0.0.0")

In [ ]:
if __name__ == "__main__":
    main()

Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Consider that you are AI Assistant named AI, AI is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics

Traceback (most recent call last):
  File "C:\local\Anaconda3\envs\py310\lib\site-packages\gradio\routes.py", line 412, in run_predict
    output = await app.get_blocks().process_api(
  File "C:\local\Anaconda3\envs\py310\lib\site-packages\gradio\blocks.py", line 1299, in process_api
    result = await self.call_function(
  File "C:\local\Anaconda3\envs\py310\lib\site-packages\gradio\blocks.py", line 1021, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\local\Anaconda3\envs\py310\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "C:\local\Anaconda3\envs\py310\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "C:\local\Anaconda3\envs\py310\lib\site-packages\anyio\_backends\_asyncio.py", line 754, in run
    result = context.run(func, *args)
  File "C:\Temp\ipykernel_10680\78328987